In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3

In [2]:
from os import walk, path
dic_csv = {}
df_var = {} 
for root,d_names,f_names in walk("./data/"):
    for f in f_names:
        filename = f.rsplit('.', 1)[0]
        dic_csv[path.join(root, f)] = ('processed_' + filename) if ('processed' in root) else filename

    

In [3]:
def make_df(dic):
    for key in dic:
        df_var[dic[key]] = pd.read_csv(key, delimiter=";")
make_df(dic_csv)

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
def separate(df, column):
    list = df[column].unique().tolist()
    dic = {}
    for l in list:
        dic[l] = df[df[column]==l]
    return dic

In [70]:
def concat_train_test(dic):
    lis = [d for d in list(dic.values()) if ('test' in d or 'train' in d)]
    train = []
    test = []
    names = []
    for l in lis:
        names.append(l.split("_")[0])
        if (l.split("_")[1] == 'train'):
            train.append(l)
        else:
            test.append(l)
    names = list(set(names))
    
    for n in names:
        t1 = [d for d in train if n in d][0]
        t2 = [d for d in test if n in d][0]
        
        df_var[n] = pd.concat([df_var[t1],df_var[t2]])
concat_train_test(dic_csv)

In [181]:
from datetime import date

def getDate(d):
    year = 1900+int(str(d)[0:2])
    month = int(str(d)[2:4])
    gender = 'M' if (month < 50) else 'F'
    month = month if (month < 50) else month - 50
    day = int(str(d)[4:6])
    return {'year': year, 'month': month, 'day': day, 'gender': gender}

def getGender(df):
    list = []
    for row in df.itertuples(index = True):
        d = getDate(row.birth_number)
        birth_number = row.birth_number 
        if (d['gender'] == 'F'):
            birth_number -= 5000
        list.append([row.client_id, birth_number, d['gender'], row.district_id])
    
    return pd.DataFrame(list, columns=['client_id', 'birth_number', 'gender', 'district_id'])

print(getGender(df_var['client']))


      client_id  birth_number gender  district_id
0             1        701213      F           18
1             2        450204      M            1
2             3        401009      F            1
3             4        561201      M            5
4             5        600703      F            5
...         ...           ...    ...          ...
5364      13955        451030      F            1
5365      13956        430406      M            1
5366      13968        680413      M           61
5367      13971        621019      F           67
5368      13998        530812      F           74

[5369 rows x 4 columns]


In [ ]:
def date_range(df,min_year, max_year):
    
    

In [203]:
def getColumns(df):
    sum = 0
    for key in df:
        if not 'train' in key and not 'test' in key:
            print('{} -> {}'.format(key, len(df_var[key].columns)))
            print('{}\n'.format(df_var[key].columns))

            sum = sum + len(df_var[key].columns)
    print(sum)

In [196]:
client = df_var['client'].copy()
disp = df_var['disp'].copy()
card = df_var['card'].copy()
loan = df_var['loan'].copy()
account = df_var['account'].copy()
card = card.rename({'type': 'card_type'}, axis=1)
disp = disp.rename({'type': 'disp_type'}, axis=1)

merge1 = pd.merge(client, disp, on ='client_id')
merge2 = pd.merge(merge1, account, on = 'account_id')
merge3 = pd.merge(merge2, card[['card_id', 'disp_id', 'card_type']], on = 'disp_id', how = 'left')
merge4 = pd.merge(loan, merge3, on = 'account_id')


In [220]:
# trans_processed = convertToDate(trans_df,'trans_train','date', False)[['account_id', 'date', 'balance']]
#    trans_processed.sort_values(by = ['account_id', 'date'])
#    t = trans_processed.groupby('account_id').tail(1)
trans = df_var['trans'].copy()
trans = trans.rename({'type': 'trans_type', 'date': 'trans_date', 'account': 'trans_account'}, axis=1)
trans.sort_values(by = ['account_id', 'trans_date'])
t = trans.groupby('account_id').tail(1)

loan_info = pd.merge(merge4, t, on = 'account_id')


,loan_id,account_id,date_x,amount_x,duration,payments,status,client_id,birth_number,district_id_x,...,card_type,trans_id,trans_date,trans_type,operation,amount_y,balance,k_symbol,bank,trans_account
321,6992,9790,960727,133968,48,2791,1.0,12050,430530,52,...,classic,2956375,960714,withdrawal,remittance to another bank,993.0,34729.5,,AB,42063897.0
396,7130,10411,961211,123408,24,5142,1.0,12810,641014,39,...,gold,3136947,961209,withdrawal,remittance to another bank,1807.0,8357.1,,GH,93566715.0


In [224]:
getColumns(loan_info)

KeyError: 'loan_id'

In [201]:
card[card['disp_id']==2166]

,card_id,disp_id,card_type,issued


In [204]:
merge4[merge4['card_type']=='classic']

,loan_id,account_id,date_x,amount,duration,payments,status,client_id,birth_number,district_id_x,disp_id,disp_type,district_id_y,frequency,date_y,card_id,card_type
40,6577,7753,940311,51696,24,2154,1.0,9593,685128,74,9285,OWNER,74,monthly issuance,930208,1005.0,classic
106,5595,3050,940902,82896,12,6908,1.0,3687,720206,49,3687,OWNER,60,monthly issuance,930508,577.0,classic
245,7243,11042,951204,217152,36,6032,1.0,13576,740707,72,13268,OWNER,72,monthly issuance,950129,1225.0,classic
295,6550,7614,960518,320592,48,6679,1.0,9427,706109,62,9119,OWNER,62,monthly issuance,950208,989.0,classic
306,5311,1766,960615,30060,60,501,1.0,2141,510906,28,2141,OWNER,28,monthly issuance,940903,337.0,classic
321,6992,9790,960727,133968,48,2791,1.0,12050,430530,52,11742,OWNER,52,monthly issuance,951109,1134.0,classic
333,7305,11359,960806,54024,12,4502,1.0,13968,680413,61,13660,OWNER,61,monthly issuance,941001,1247.0,classic
337,5088,544,960822,91152,24,3798,1.0,655,421210,32,655,OWNER,32,monthly issuance,950515,119.0,classic
438,6938,9504,970310,341280,60,5688,NaN,11701,490414,9,11393,OWNER,9,monthly issuance,960129,1118.0,classic
488,7119,10361,970602,86880,12,7240,NaN,12750,405525,21,12442,OWNER,21,monthly issuance,960621,1173.0,classic


In [207]:
card[card['card_id']==17]

,card_id,disp_id,card_type,issued
21,17,127,classic,980607


<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<ipython-input-158-8140e5144396>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  530812[1]


TypeError: 'int' object is not subscriptable

In [ ]:
def db():
    CSV_files = {'account': account, 'card': card_train, 'client': client, 'disp': disp, 'disctrict': district, 'loan': loan_train, 'trans': trans_train}
    conn = sqlite3.connect('database.db')
    # Correr só uma vez
    if not os.path.exists('./database.db'):
        for f in CSV_files:
            CSV_files[f].to_sql(f, conn)
    query = """
        SELECT loan_id, loan.account_id, status
        FROM loan, account
        WHERE loan.account_id == account.account_id
        """
         
    pd.read_sql_query(query, conn)
    
def getCorr(df, size=(11,9)):
    corr = df.corr()
    ax = plt.subplots(figsize=size)
    ax = sns.heatmap(corr,  annot = True, linewidths=.1, mask = np.triu(corr), cmap='coolwarm')

    def checkUniqueCategoricalVars():
    for f in CSV_files:
        print("{}:\n {}\n\n".format(f, CSV_files[f].select_dtypes(include=['object']).nunique()))